In [150]:
# from torch.utils.data import DataLoader

import torchtext
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
from torchtext.data import get_tokenizer

import torch
from torch.utils.data import DataLoader
from torch.nn import ReLU, Sigmoid, Sequential, Linear, BCELoss
from torch.optim import Adam

import nltk
from nltk.corpus import stopwords

import re
from string import punctuation
import time

In [2]:
train_dataset, eval_dataset = torchtext.datasets.IMDB()

In [3]:
train_dataset, eval_dataset = to_map_style_dataset(train_dataset), to_map_style_dataset(eval_dataset)

In [16]:
for label,text in train_dataset[:10]:
  print(label, text)

1 I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, e

In [23]:
tokenizer = get_tokenizer('basic_english')
tokenizer(train_dataset[0][1])[:5]

['i', 'rented', 'i', 'am', 'curious-yellow']

In [51]:
try:
  stopwords.words('english')[:5]
except LookupError:
  nltk.download('stopwords')
finally:
  en_stop_words = set(stopwords.words('english'))

In [39]:
re_punc = re.compile('[%s]' % re.escape(punctuation))

In [55]:
def filter_tokens(tokens):
  tokens = [token.lower() for token in tokens]
  tokens = [re_punc.sub('', token) for token in tokens]
  tokens = [token for token in tokens if token.isalpha()]
  tokens = [token for token in tokens if token not in en_stop_words]
  tokens = [token for token in tokens if len(token) > 1]
  return tokens

In [57]:
for _, text in train_dataset[:10]:
  print(filter_tokens(tokenizer(text)))

['rented', 'curiousyellow', 'video', 'store', 'controversy', 'surrounded', 'first', 'released', 'also', 'heard', 'first', 'seized', 'customs', 'ever', 'tried', 'enter', 'country', 'therefore', 'fan', 'films', 'considered', 'controversial', 'really', 'see', 'plot', 'centered', 'around', 'young', 'swedish', 'drama', 'student', 'named', 'lena', 'wants', 'learn', 'everything', 'life', 'particular', 'wants', 'focus', 'attentions', 'making', 'sort', 'documentary', 'average', 'swede', 'thought', 'certain', 'political', 'issues', 'vietnam', 'war', 'race', 'issues', 'united', 'states', 'asking', 'politicians', 'ordinary', 'denizens', 'stockholm', 'opinions', 'politics', 'sex', 'drama', 'teacher', 'classmates', 'married', 'men', 'kills', 'curiousyellow', 'years', 'ago', 'considered', 'pornographic', 'really', 'sex', 'nudity', 'scenes', 'far', 'even', 'shot', 'like', 'cheaply', 'made', 'porno', 'countrymen', 'mind', 'find', 'shocking', 'reality', 'sex', 'nudity', 'major', 'staple', 'swedish', 'ci

In [58]:
def build_vocabulary(datasets):
  for dataset in datasets:
    for _, text in dataset:
      yield filter_tokens(tokenizer(text))

In [95]:

min_freq = 3

vocabulary = build_vocab_from_iterator(
  build_vocabulary([train_dataset, eval_dataset]), 
  min_freq=min_freq, 
  specials=["<UNK>"]  
)

vocabulary.set_default_index(vocabulary["<UNK>"]) 

In [93]:
len(vocabulary)

53176

In [96]:
# get the word of index 100
vocabulary["henrique"]

0

In [98]:
torch.save(vocabulary, 'data/vocab-cnn.pth')

In [101]:
def encode_bow(vocab, text):
  encoded_text = torch.zeros(len(vocab), dtype=torch.float32)
  text = filter_tokens(tokenizer(text))
  token_ids = vocab(text)

  for token_id in token_ids:
    if token_id:
      encoded_text[token_id] += 1.0

  return encoded_text

In [102]:
encode_bow(vocabulary, train_dataset[10][1])

tensor([0., 2., 2.,  ..., 0., 0., 0.])

In [108]:
train_dataset[10][1]

'It was great to see some of my favorite stars of 30 years ago including John Ritter, Ben Gazarra and Audrey Hepburn. They looked quite wonderful. But that was it. They were not given any characters or good lines to work with. I neither understood or cared what the characters were doing.<br /><br />Some of the smaller female roles were fine, Patty Henson and Colleen Camp were quite competent and confident in their small sidekick parts. They showed some talent and it is sad they didn\'t go on to star in more and better films. Sadly, I didn\'t think Dorothy Stratten got a chance to act in this her only important film role.<br /><br />The film appears to have some fans, and I was very open-minded when I started watching it. I am a big Peter Bogdanovich fan and I enjoyed his last movie, "Cat\'s Meow" and all his early ones from "Targets" to "Nickleodeon". So, it really surprised me that I was barely able to keep awake watching this one.<br /><br />It is ironic that this movie is about a de

In [112]:
vocabulary.get_stoi()['films']

29

In [107]:
vocabulary.get_itos()[1], vocabulary.get_itos()[2]

('movie', 'film')

In [113]:
for i in range(0, 10):
  print(encode_bow(vocabulary, train_dataset[i][1]))

tensor([0., 0., 3.,  ..., 0., 0., 0.])
tensor([0., 0., 2.,  ..., 0., 0., 0.])
tensor([0., 0., 3.,  ..., 0., 0., 0.])
tensor([0., 1., 5.,  ..., 0., 0., 0.])
tensor([0., 0., 7.,  ..., 0., 0., 0.])
tensor([0., 0., 1.,  ..., 0., 0., 0.])
tensor([0., 2., 1.,  ..., 0., 0., 0.])
tensor([0., 2., 0.,  ..., 0., 0., 0.])
tensor([0., 6., 1.,  ..., 0., 0., 0.])
tensor([0., 1., 2.,  ..., 0., 0., 0.])


In [130]:
def encode_batch_text(batch):
  labels, texts = list(zip(*batch))
  encoded_texts = [encode_bow(vocabulary, text) for text in texts]
  return torch.stack(encoded_texts), torch.tensor(labels, dtype=torch.float32) - 1

In [124]:
encode_batch_text(train_dataset[:4], vocabulary)

(tensor([[0., 0., 3.,  ..., 0., 0., 0.],
         [0., 0., 2.,  ..., 0., 0., 0.],
         [0., 0., 3.,  ..., 0., 0., 0.],
         [0., 1., 5.,  ..., 0., 0., 0.]]),
 tensor([0., 0., 0., 0.]))

In [131]:
train_loader = DataLoader(train_dataset, batch_size=512, collate_fn=encode_batch_text, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=512, collate_fn=encode_batch_text)

In [132]:
for encoded_text, encoded_labels in train_loader:
 print(encoded_text)
 print(encoded_labels) 

tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  7.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  7.,  ...,  0.,  0.,  0.],
        [ 0.,  0., 10.,  ...,  0.,  0.,  0.],
        [ 0.,  2.,  2.,  ...,  0.,  0.,  0.]])
tensor([0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1.,
        0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0.,
        1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0.,
        1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0.,
        0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1.,
        0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
        1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1.,
        1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.

In [134]:


def define_model(n_words):
  model = Sequential(
    Linear(in_features=n_words, out_features=1024),
    ReLU(),
    Linear(in_features=1024, out_features=50),
    ReLU(),
    Linear(in_features=50, out_features=1),
    Sigmoid()
  )

  return model

In [136]:
model = define_model(len(vocabulary))
print(model)

Sequential(
  (0): Linear(in_features=53176, out_features=1024, bias=True)
  (1): ReLU()
  (2): Linear(in_features=1024, out_features=50, bias=True)
  (3): ReLU()
  (4): Linear(in_features=50, out_features=1, bias=True)
  (5): Sigmoid()
)


In [139]:
bows, labels = encode_batch_text(train_dataset[:4])
print(model(bows))

for p in model.parameters():
  print(p)

tensor([[0.4724],
        [0.4701],
        [0.4702],
        [0.4701]], grad_fn=<SigmoidBackward0>)
Parameter containing:
tensor([[-9.2113e-04, -5.7925e-04,  1.0131e-03,  ..., -2.8549e-03,
         -2.8531e-03,  2.8559e-03],
        [ 2.1938e-03, -3.1650e-03, -2.2208e-03,  ..., -1.8708e-03,
          1.7883e-03, -3.7103e-03],
        [ 3.3647e-03, -3.4460e-03, -1.1352e-03,  ..., -2.9820e-03,
          2.5997e-05, -1.5544e-03],
        ...,
        [ 2.9458e-03, -2.0241e-03, -2.4434e-03,  ...,  3.1770e-03,
          1.4586e-03,  4.1633e-03],
        [-4.1034e-03,  3.4763e-04,  1.8419e-03,  ..., -2.1104e-03,
          2.4688e-03, -1.7886e-03],
        [ 2.7585e-03, -1.5078e-03,  3.6700e-03,  ..., -1.8403e-03,
         -3.1584e-03, -2.4770e-03]], requires_grad=True)
Parameter containing:
tensor([ 0.0012,  0.0028, -0.0014,  ...,  0.0012,  0.0013,  0.0040],
       requires_grad=True)
Parameter containing:
tensor([[ 0.0070,  0.0187, -0.0232,  ...,  0.0211, -0.0201, -0.0223],
        [ 0.022

In [141]:
learning_rate = 1e-3
optimizer = Adam(model.parameters(), lr=learning_rate)
loss_fn = BCELoss() 

In [142]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
loss_fn = loss_fn.to(device) 

In [ ]:
import time

In [145]:
def train(model, iterator, optimizer, criterion):
  epoch_loss = 0
  epoch_acc = 0

  model.train()

  for bows, labels in iterator:
    bows = bows.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()

    predictions = model(bows).squeeze(1)

    loss = criterion(predictions, labels)

    rounded_preds = torch.round(predictions)
    correct = (rounded_preds == labels).float()
    acc = correct.sum() / len(correct)

    loss.backward()

    optimizer.step()

    epoch_loss += loss.item()
    epoch_acc += acc.item()

  return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [144]:
def validate(model, iterator, criterion):
  epoch_loss = 0
  epoch_acc = 0

  model.eval()

  with torch.no_grad():
    for bows, labels in iterator:
      bows = bows.to(device)
      labels = labels.to(device)

      predictions = model(bows).squeeze(1)

      loss = criterion(predictions, labels)

      rounded_preds = torch.round(predictions)
      correct = (rounded_preds == labels).float()
      acc = correct.sum() / len(correct)

      epoch_loss += loss.item()
      epoch_acc += acc.item()

  return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [146]:
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))

  return elapsed_mins, elapsed_secs

In [147]:
from tqdm.auto import tqdm
num_training_steps = 1000
progress_bar = tqdm(range(num_training_steps))

/Users/hchaves/Github/IMT/imt-imda-course/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/1000 [00:00<?, ?it/s]

In [152]:
N_EPOCHS = 10
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
  start_time = time.time()
  train_loss, train_acc = train(model, train_loader, optimizer, loss_fn)
  valid_loss, valid_acc = validate(model, eval_loader, loss_fn)
  end_time = time.time()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), 'data/sentiment-model-1.pt')
  
  print(f'Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
  print(f'\t Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc * 100:.2f}%')

Epoch: 01 | Epoch Time: 0m 40s
	Train Loss: 0.262 | Train Acc: 90.49%
	 Val. Loss: 0.336 | Val. Acc: 86.78%
Epoch: 02 | Epoch Time: 0m 40s
	Train Loss: 0.094 | Train Acc: 97.22%
	 Val. Loss: 0.566 | Val. Acc: 85.85%
Epoch: 03 | Epoch Time: 0m 41s
	Train Loss: 0.039 | Train Acc: 99.02%
	 Val. Loss: 0.757 | Val. Acc: 85.60%
Epoch: 04 | Epoch Time: 0m 41s
	Train Loss: 0.017 | Train Acc: 99.64%
	 Val. Loss: 0.927 | Val. Acc: 85.11%
Epoch: 05 | Epoch Time: 0m 39s
	Train Loss: 0.008 | Train Acc: 99.93%
	 Val. Loss: 1.175 | Val. Acc: 84.97%
Epoch: 06 | Epoch Time: 0m 40s
	Train Loss: 0.005 | Train Acc: 99.99%
	 Val. Loss: 1.238 | Val. Acc: 85.21%
Epoch: 07 | Epoch Time: 0m 40s
	Train Loss: 0.005 | Train Acc: 99.99%
	 Val. Loss: 1.309 | Val. Acc: 85.05%
Epoch: 08 | Epoch Time: 0m 41s
	Train Loss: 0.004 | Train Acc: 100.00%
	 Val. Loss: 1.389 | Val. Acc: 85.07%
Epoch: 09 | Epoch Time: 0m 40s
	Train Loss: 0.004 | Train Acc: 100.00%
	 Val. Loss: 1.444 | Val. Acc: 85.06%
Epoch: 10 | Epoch Time: 0m

In [158]:
model(torch.stack([encode_bow(vocabulary, "This movie is amazing")])).item()

0.9315168261528015